# Early-life exposure to butterbeer and later-life health consequences
This workshop will take you through a synthetic dataset to identify associations between early-life exposure to butterbeer and health in later-life.

On Jan 1, 1954, the UK government ended a 10-year long ban on butterbeer. That means that everyone born before Jan 1954 was likely to not be exposed to maternal butterbeer consumption, while those born after Jan 1954 were exposed to maternal butterbeer consumption in utero.

The cohort you have is a UK Biobank-like cohort. Individuals in this specific one were born between 1952-1955. Participants filled out several questionnaires. You have data on their early life conditions (DOB, where they were born, sex, their income). We also have their medical records and know if they were diagnosed with chronic disease (hypertension and diabetes, for example). In addition, we have blood samples from when they were ~50 years old and have their normalised plasma protein expression levels. We also have some other variables (while may not seem relevant, could be) such as how many car accidents they've been in.

There are 5 actionable tasks in this notebook. They might require you to write/edit code or interpret results (which you can do on a piece of paper). Most of the code is otherwise written for you. If you'd like to challenge yourself, you can explore the dataset yourself (in the Github) and analyse.

## Load packages into R

In [ ]:
### Install and load these packages into R. They have different functions that we will use throughout this exercise.
install.packages(c("broom", "dplyr", "ggplot2", "gtsummary", "knitr", "purrr"))
library(broom)
library(dplyr)
library(ggplot2)
library(gtsummary)
library(knitr)
library(purrr)

## Read in data and clean

In [ ]:
### Read in data from your computer into R
df = read.csv("butterbeer.csv", header = TRUE)

In [ ]:
### Visualise the first few rows of the dataframe. This is a good opportunity for us to look
### at the variables we have, the type of variables they are, and other features
head(df)

In [ ]:
### This next block cleans the dataframe a bit. You don't need to look at it too closely

## Fix DOB variable
df$dob = as.Date(df$dob, format = "%d-%b-%y")

## Fix century (because %y makes 55 → 2055)
df$dob[df$dob > Sys.Date()] = df$dob[df$dob > Sys.Date()] - 36525

## Compute age
df$age = as.integer(floor((Sys.Date() - df$dob) / 365.25))


#### Task 1
You'll need to make an exposure variable based off the years of birth. In this cohort, individuals were born 1952-1955. Based off the introduction of the ban, what birth years are in the unexposed group and exposed group? Fill in XXX with the correct dates in the cell below. For the purpose of this, we will say exposed means exposed to excess butterbeer.

In [ ]:
## Make an exposure variable.
## We do this by assinging everyone who was born in XXX and XXX to 
## the "EXPOSED" group and those born in XXX and XXX to the "UNEXPOSED" group

df$birth_year <- as.integer(format(df$dob, "%Y"))

df$exposure[df$birth_year %in% c(XXX, XXX)] <- "unexposed"
df$exposure[df$birth_year %in% c(XXX, XXX)] <- "exposed"

In [ ]:
## Make categorical variables into factors
df = df %>%
  mutate(
    sex = factor(sex),
    location = factor(location),
    diabetes_ever = factor(diabetes_ever),
    hypertension_ever = factor(hypertension_ever),
    exposure = factor(exposure, levels = c("unexposed", "exposed"))
  )

In [ ]:
### Look at a summary of the data
summary(df)

## Analyses

### Descriptives

Let's explore how this exposure varies by different characteristics

In [ ]:
# This creates a Table 1 (table of descriptive characteristics by exposure status)
table1 = tbl_summary(
  data = df,            
  by = exposure,     
  include = c(age, sex, location, income, car_accidents, diabetes_ever, hypertension_ever,
              protein_1, protein_2, protein_3, protein_4, protein_5, protein_6, protein_7
             ),
    statistic = all_continuous() ~ "{mean} ({sd})"
)

In [ ]:
# Visualise Table 1
table1 %>% as_kable()

#### Task 2
Change y = [sex] to all the other proteins and outcomes to see how they differ visually. What trends do you notice?

In [ ]:
### Further visualisations
ggplot(df, aes(x = exposure, y = sex)) +
  geom_boxplot() +
  geom_jitter(width = 0.1, alpha = 0.5) +
  theme_minimal(base_size = 14)

### Associations

#### Task 3
Let's take a look at Table 1 again and make sure we have the correct population. Is there a group of people that should not be included in this? Fill in the XXX below (hint it's a location).

In [ ]:
df_analysis = df %>%
  filter(location != "XXX")

Let's now look at the association between exposure and different later-life outcomes.

You'll see we are running a logistic regression model to look at the association between the exposure and diabetes/hypertension. I've included sex, location and income as covariates.

In [ ]:
model_diabetes = glm(diabetes_ever ~ exposure + sex + location + income,
             data = df_analysis,
             family = binomial)

# tidy results
diabetes_result <- tidy(model_diabetes, conf.int = TRUE, exponentiate = TRUE) %>%
  filter(term == "exposureexposed")

# forest plot
ggplot(diabetes_result, aes(y = term, x = estimate, xmin = conf.low, xmax = conf.high)) +
  geom_point() +
  geom_errorbarh(height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed") +
  xlab("Odds Ratio") +
  ylab("") +
  theme_minimal(base_size = 14)

In [ ]:
model_hypertension = glm(hypertension_ever ~ exposure + sex + location + income,
             data = df_analysis,
             family = binomial)

# tidy results
hypertension_result <- tidy(model_hypertension, conf.int = TRUE, exponentiate = TRUE) %>%
  filter(term == "exposureexposed")

# forest plot
ggplot(hypertension_result, aes(y = term, x = estimate, xmin = conf.low, xmax = conf.high)) +
  geom_point() +
  geom_errorbarh(height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed") +

  xlab("Odds Ratio") +
  ylab("") +
  theme_minimal(base_size = 14)

#### Task 4
How would you interpret the above odds ratios?

A few more associations to look at (now linear regression). I've listed all of them below.

In [ ]:
model_protein1 = lm(protein_1 ~ exposure + sex + income + location, data = df_analysis)
model_protein2 = lm(protein_2 ~ exposure + sex + income + location, data = df_analysis)
model_protein3 = lm(protein_3 ~ exposure + sex + income + location, data = df_analysis)
model_protein4 = lm(protein_4 ~ exposure + sex + income + location, data = df_analysis)
model_protein5 = lm(protein_5 ~ exposure + sex + income + location, data = df_analysis)
model_protein6 = lm(protein_6 ~ exposure + sex + income + location, data = df_analysis)
model_protein7 = lm(protein_7 ~ exposure + sex + income + location, data = df_analysis)
model_car = lm(car_accidents ~ exposure, data = df_analysis)

You'll need to update model_protein1 in the two cells below to see how the different outcomes differ by exposure.

In [ ]:
# This prints out the summary of the results while the next cell will visualise the same results.
summary(model_protein1)

In [ ]:
# tidy results
result <- tidy(model_protein1, conf.int = TRUE) %>%
  filter(term == "exposureexposed")

# forest plot
ggplot(result, aes(y = term, x = estimate, xmin = conf.low, xmax = conf.high)) +
  geom_point() +
  geom_errorbarh(height = 0.2) +
  geom_vline(xintercept = 0, linetype = "dashed") +
  xlab("Beta Coefficient") +
  ylab("") +
  theme_minimal(base_size = 14)

#### Task 5
How would you interpret the above beta coefficients?

### Conclusion

That's it! Good job! 

What is your conclusion from the above analyses? Did in utero exposure to butterbear impact health in later life? What are some strengths and limitations?